In [7]:
import random

TARGET = "CHISPAS"
POPULATION_SIZE = 100
MUTATION_RATE = 0.01
GENERATIONS = 500

In [789]:
import random

TARGET = "CHISPAS"
POPULATION_SIZE = 100
MUTATION_RATE = 0.25
GENERATIONS = 500
PROBABILIDAD_CRUCE = 0.7
NUM_PADRES = 50
TOUR_SIZE = 5

# creamos individuo
def create_individual(length):
    return ''.join(random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') for i in range(length))

#funcion fitness 
def fitness(individual):
    matches = sum(1 for a, b in zip(individual, TARGET) if a == b)
    return matches / len(TARGET)

#seleccionamos la poblacion
def selection(population, method='elite'):
    if method == 'elite':
        population = sorted(population, key=lambda ind: fitness(ind), reverse=True)
        return population[:2]
    elif method == 'ruleta':
        # calculamos la probabilidad de seleccion para cada individuo
        total_fitness = sum(fitness(ind) for ind in population)
        selection_probs = [fitness(ind) / total_fitness for ind in population]
        # seleccionamos n (osea 4 padres) aleatoriamente usando las probabilidades
        selected = random.choices(population, weights=selection_probs, k=NUM_PADRES)
        return selected
    elif method == 'aptitud':
        population = sorted(population, key=lambda ind: fitness(ind), reverse=True)
        top_half = population[:len(population) // 2]
        return random.sample(top_half, k=NUM_PADRES)
    elif method == 'muestreo_estocastico':
        total_fitness = sum(fitness(ind) for ind in population)
        selection_probs = [fitness(ind) / total_fitness for ind in population]
        distance = 1.0 / NUM_PADRES  # Distribuir uniformemente
        start_point = random.uniform(0, distance)
        points = [start_point + i * distance for i in range(NUM_PADRES)]
        
        selected = []
        cumulative_prob = 0
        j = 0  # Controla el índice de puntos
        
        for ind, prob in zip(population, selection_probs):
            cumulative_prob += prob
            while j < NUM_PADRES and cumulative_prob > points[j]:
                selected.append(ind)
                j += 1
                if j >= NUM_PADRES:
                    break  # Salir si ya tenemos suficientes padres seleccionados
        #print(len(selected))  # Para verificar que siempre selecciona el número correcto de padres
        return selected
    elif method == 'torneo':
        selected = []
        for _ in range(NUM_PADRES):
            tournament = random.sample(population, TOUR_SIZE)
            winner = max(tournament, key=fitness)
            selected.append(winner)
        return selected
    elif method == 'rango':
        population = sorted(population, key=lambda ind: fitness(ind))
        rank_weights = [i + 1 for i in range(len(population))]
        total_weight = sum(rank_weights)
        selection_probs = [weight / total_weight for weight in rank_weights]
        return random.choices(population, weights=selection_probs, k=NUM_PADRES)
    
#cruce de padres para crear un hijo
def crossover(parent1, parent2, method='uniforme'):
    if method == 'uniforme':
        child = ''.join(parent1[i] if random.random() > 0.5 else parent2[i] for i in range(len(parent1)))
    elif method == 'un_punto':
        punto = random.randint(1, len(parent1) - 1)
        child = parent1[:punto] + parent2[punto:]
    elif method == 'dos_puntos':
        punto1, punto2 = sorted(random.sample(range(1, len(parent1)), 2))
        child = parent1[:punto1] + parent2[punto1:punto2] + parent1[punto2:]
    elif method == 'punto_medio':
        punto_medio = len(parent1) // 2
        child = parent1[:punto_medio] + parent2[punto_medio:]
    return child

def mutate(individual, method='punto'):
    if method == 'punto':
        return ''.join(random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') if random.random() < MUTATION_RATE else char for char in individual)
    elif method == 'inversion':
        if random.random() < MUTATION_RATE:
            i, j = sorted(random.sample(range(len(individual)), 2))
            individual = individual[:i] + individual[i:j][::-1] + individual[j:]
        return individual
    elif method == 'permutacion':
        if random.random() < MUTATION_RATE:
            i, j = sorted(random.sample(range(len(individual)), 2))
            individual = list(individual)
            individual[i], individual[j] = individual[j], individual[i]
            individual = ''.join(individual)
        return individual
    elif method == 'adaptativa':
        adaptative_rate = MUTATION_RATE / (1 + fitness(individual))
        return ''.join(random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ') if random.random() < adaptative_rate else char for char in individual)
    elif method == 'uniforme':
        mutated = ""
        for char in individual:
            if random.random() < MUTATION_RATE:
                mutated += random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
            else:
                mutated += char
        return mutated

def genetic_algorithm(selection_method='elite', crossover_method='uniforme', mutation_method='punto'):
    #creas poblacion inicial
    population = [create_individual(len(TARGET)) for i in range(POPULATION_SIZE)]
    solutions = []
    for generation in range(GENERATIONS):
        best_individual = max(population, key=fitness)
        solutions.append(fitness(best_individual))
        
        
        #condicion de finalizacion modificada
        #if(abs(fitness(best_individual) - best_solution) < epsilon):
        #    print(f" Condicion: {abs(fitness(best_individual)  - best_solution)}")
        #    break
        #else:
        #    best_solution = fitness(best_individual)

        print(f"Generacion {generation}: Mejor individuo = {best_individual}, Fitness = {fitness(best_individual):.2f}")

        if best_individual == TARGET:
            print(f"!Enconrado en generacion {generation}")
            break

        new_population = []
        #print(f"Mi poblacion es: {[fitness(e) for e in population]}")
        #parent1, parent2 = selection(population)
        parents = selection(population, method=selection_method)
        #print(f"padres son: {parent1} y {parent2}")
        for _ in range(POPULATION_SIZE // len(parents)):
            for i in range(0, len(parents), 2):
                if random.random() < PROBABILIDAD_CRUCE:
                    child1 = mutate(crossover(parents[i], parents[i+1], method=crossover_method), method=mutation_method)
                    child2 = mutate(crossover(parents[i], parents[i+1], method=crossover_method), method=mutation_method)
                else:
                    child1 = mutate(parents[i])
                    child2 = mutate(parents[i])
                new_population.extend([child1, child2])
        #print(f"Hijos son {new_population}")
        population = new_population
        #print(f"Population size is {len(population)}")
    #print(solutions)

In [1003]:
genetic_algorithm(selection_method='torneo', crossover_method='uniforme', mutation_method='adaptativa')

Generacion 0: Mejor individuo = YVIKYAX, Fitness = 0.29
Generacion 1: Mejor individuo = MCISSJO, Fitness = 0.29
Generacion 2: Mejor individuo = CHXIFJS, Fitness = 0.43
Generacion 3: Mejor individuo = NHISPHK, Fitness = 0.57
Generacion 4: Mejor individuo = CHISPZX, Fitness = 0.71
Generacion 5: Mejor individuo = CHISPHS, Fitness = 0.86
Generacion 6: Mejor individuo = CHISPHS, Fitness = 0.86
Generacion 7: Mejor individuo = CHISPNS, Fitness = 0.86
Generacion 8: Mejor individuo = CHISPAS, Fitness = 1.00
!Enconrado en generacion 8


In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.ndimage import convolve, sobel
from skimage.metrics import structural_similarity as ssim

# Parámetros del algoritmo genético
TAMANO_POBLACION = 10
GENERACIONES = 500
TASA_MUTACION = 0.1
PROBABILIDAD_CRUCE = 0.85
NUM_PADRES = 4  # Número de padres que seleccionaremos
TOUR_SIZE = 3   # Tamaño para selección por torneo

# Cargar y preprocesar la imagen médica
radiografia = Image.open("sebas.jpg").convert("L").resize((256, 256))
radiografia = np.array(radiografia) / 255.0

# Función para crear un filtro inicial
def crear_individuo():
    return np.random.uniform(0, 1, (3, 3))

# Función para aplicar un filtro a una imagen
def aplicar_filtro(imagen, filtro):
    return convolve(imagen, filtro)

# Función para calcular el borde usando el filtro Sobel
def calcular_borde(imagen):
    grad_x = sobel(imagen, axis=0)
    grad_y = sobel(imagen, axis=1)
    return np.hypot(grad_x, grad_y)

# Función de fitness con penalización por pérdida de bordes
def fitness(individuo):
    imagen_filtrada = aplicar_filtro(radiografia, individuo)
    ssim_value = ssim(radiografia, imagen_filtrada, data_range=imagen_filtrada.max() - imagen_filtrada.min())
    bordes_original = calcular_borde(radiografia)
    bordes_filtrados = calcular_borde(imagen_filtrada)
    perdida_bordes = np.mean((bordes_original - bordes_filtrados) ** 2)
    return ssim_value - 0.5 * perdida_bordes

#
#def fitness(individuo):
#    imagen_filtrada = aplicar_filtro(radiografia, individuo)
#    # ssim_value = ssim(radiografia, imagen_filtrada, data_range=imagen_filtrada.max() - imagen_filtrada.min())
#    # # psnr_value = psnr(radiografia, imagen_filtrada)
#    mse = np.mean(abs(radiografia - imagen_filtrada))
#    # print(mse)
#    if mse == 0:
#        psnr_value = float('inf')
#    else:
#        MAX = 1.0
#        psnr_value = 10 * np.log10(MAX**2 / mse)
#    return psnr_value
#    # return ssim_value
#
# Métodos de selección
def seleccion(poblacion, method='elite'):
    if method == 'elite':
        poblacion = sorted(poblacion, key=lambda x: fitness(x), reverse=True)
        return poblacion[:NUM_PADRES]
    
    elif method == 'ruleta':
        total_fitness = sum(fitness(ind) for ind in poblacion)
        selection_probs = [fitness(ind) / total_fitness for ind in poblacion]
        return random.choices(poblacion, weights=selection_probs, k=NUM_PADRES)
    
    elif method == 'aptitud':
        poblacion = sorted(poblacion, key=lambda x: fitness(x), reverse=True)
        top_half = poblacion[:len(poblacion) // 2]
        return random.sample(top_half, k=NUM_PADRES)
    
    elif method == 'muestreo_estocastico':
        total_fitness = sum(fitness(ind) for ind in poblacion)
        selection_probs = [fitness(ind) / total_fitness for ind in poblacion]
        distance = 1.0 / NUM_PADRES
        start_point = random.uniform(0, distance)
        points = [start_point + i * distance for i in range(NUM_PADRES)]
        selected = []
        cumulative_prob = 0
        j = 0
        for ind, prob in zip(poblacion, selection_probs):
            cumulative_prob += prob
            while j < NUM_PADRES and cumulative_prob > points[j]:
                selected.append(ind)
                j += 1
        return selected

    elif method == 'torneo':
        selected = []
        for _ in range(NUM_PADRES):
            tournament = random.sample(poblacion, TOUR_SIZE)
            winner = max(tournament, key=fitness)
            selected.append(winner)
        return selected
    
    elif method == 'rango':
        poblacion = sorted(poblacion, key=lambda ind: fitness(ind))
        rank_weights = [i + 1 for i in range(len(poblacion))]
        total_weight = sum(rank_weights)
        selection_probs = [weight / total_weight for weight in rank_weights]
        return random.choices(poblacion, weights=selection_probs, k=NUM_PADRES)

# Métodos de cruce
def crossover(parent1, parent2, method='uniforme'):
    if method == 'uniforme':
        mask = np.random.randint(0, 2, (3, 3)).astype(bool)
        child = np.copy(parent1)
        child[mask] = parent2[mask]
    elif method == 'un_punto':
        punto = random.randint(1, 2)
        child = np.copy(parent1)
        child[:punto, :] = parent2[:punto, :]
    elif method == 'dos_puntos':
        punto1, punto2 = sorted(random.sample(range(3), 2))
        child = np.copy(parent1)
        child[punto1:punto2, :] = parent2[punto1:punto2, :]
    elif method == 'punto_medio':
        child = np.copy(parent1)
        child[1, :] = parent2[1, :]
    return child

# Métodos de mutación
def mutacion(individuo, method='simple'):
    mutante = np.copy(individuo)
    if method == 'simple':
        for i in range(mutante.shape[0]):
            for j in range(mutante.shape[1]):
                if random.random() < TASA_MUTACION:
                    mutante[i, j] = np.random.uniform(0, 1)
    elif method == 'swap':
        i1, j1, i2, j2 = random.sample(range(3), 4)
        mutante[i1, j1], mutante[i2, j2] = mutante[i2, j2], mutante[i1, j1]
    elif method == 'crecimiento':
        for i in range(3):
            for j in range(3):
                if random.random() < TASA_MUTACION:
                    mutante[i, j] *= np.random.uniform(0.8, 1.2)
                    mutante[i, j] = max(0, min(1, mutante[i, j]))
    elif method == 'reduccion':
        for i in range(3):
            for j in range(3):
                if random.random() < TASA_MUTACION:
                    mutante[i, j] -= np.random.uniform(0, 0.5)
                    mutante[i, j] = max(0, mutante[i, j])
    return mutante

# Algoritmo Genético Principal
def algoritmo_genetico(selection_method='elite', crossover_method='uniforme', mutation_method='simple'):
    poblacion = [crear_individuo() for _ in range(TAMANO_POBLACION)]
    for generacion in range(GENERACIONES):
        padres = seleccion(poblacion, method=selection_method)
        nueva_poblacion = []
        for _ in range(TAMANO_POBLACION // len(padres)):
            for i in range(0, len(padres), 2):
                if random.random() < PROBABILIDAD_CRUCE:
                    child1 = mutacion(crossover(padres[i], padres[i+1], method=crossover_method), method=mutation_method)
                    child2 = mutacion(crossover(padres[i], padres[i+1], method=crossover_method), method=mutation_method)
                else:
                    child1 = mutacion(padres[i], method=mutation_method)
                    child2 = mutacion(padres[i+1], method=mutation_method)
                nueva_poblacion.extend([child1, child2])
        poblacion = nueva_poblacion
        # Monitorear progreso
        if generacion % 50 == 0:
            mejor = max(poblacion, key=fitness)
            print(f"Generación {generacion}, Fitness: {fitness(mejor)}")
            #imagen_filtrada = aplicar_filtro(radiografia, mejor)
            #plt.imshow(imagen_filtrada, cmap='gray')
            #plt.title(f"Generación {generacion}")
            #plt.show()
    mejor = max(poblacion, key=fitness)
    return mejor

# Ejecutar el algoritmo genético
mejor_filtro = algoritmo_genetico(selection_method='torneo', crossover_method='dos_puntos', mutation_method='crecimiento')


imagen_final = aplicar_filtro(radiografia, mejor_filtro)
plt.imshow(imagen_final, cmap='gray')
plt.title("Radiografía Mejorada")
plt.show()

# Aplicar el mejor filtro a la imagen original y mostrarla
imagen_final = aplicar_filtro(radiografia, mejor_filtro)
plt.imshow(imagen_final, cmap='gray')
plt.title("Radiografía Mejorada")
plt.show()
